In [ ]:
import tensorflow as tf
import tensorflow.examples.tutorials.mnist.input_data as input_data

In [ ]:
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

## 建立模型

In [ ]:
# define the fullly connect network
def fcn_layer(inputs,
             input_dim,
             output_dim,
             activation=None):
    W = tf.Variable(tf.truncated_normal([input_dim, output_dim], stddev=0.1))
    b = tf.Variable(tf.zeros([output_dim]))
    XWB = tf.matmul(inputs, W) + b
    if activation == None:
        return XWB
    return activation(XWB)    

## 构建输入层

In [ ]:
x = tf.placeholder(tf.float32, [None, 28*28], name="X")

## 构建隐藏层

In [ ]:
# hidden layer contains 256 nueral
h1=fcn_layer(inputs=x,
             input_dim=28*28,
             output_dim=256,
             activation=tf.nn.relu)


## 构建输出层

In [ ]:
forward = fcn_layer(inputs=h1,
             input_dim=256,
             output_dim=10,
             activation=None)

pred = tf.nn.softmax(forward)

## 训练模型
### 定义标签

In [ ]:
y = tf.placeholder(tf.float32, [None, 10], name = "Y")

### 定义损失函数

In [ ]:
loss_function = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=forward))

### 设置训练参数

In [ ]:
train_epochs = 40
batch_size = 50
total_batch = mnist.train.num_examples // batch_size
display_step = 1
learning_rate = 0.01

### 设置优化器

In [ ]:
optimizer = tf.train.AdamOptimizer(learning_rate).minimize(loss_function)

### 定义准确率

In [ ]:
correct_predict = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32))

###  训练模型

In [ ]:
from time import time
startTime = time()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for epoch in range(train_epochs):
        for batch in range(total_batch):
            xs, ys = mnist.train.next_batch(batch_size)
            sess.run(optimizer, feed_dict={x: xs, y: ys})
            
        loss, acc=sess.run([loss_function, accuracy],
                           feed_dict={x:mnist.validation.images,
                                     y:mnist.validation.labels})
        
        if epoch % display_step == 0:
            print("epoch: %02d" % epoch, 
                  "Loss: {:.9f}".format(loss),
                  "Acc: {:.9f}".format(acc))